# Determining Optimal Gate Bias 

In [1]:
import qcodes as qc
import numpy as np
from time import sleep

import sweep 
from sweep.sweep_load import pload1d
from barreralabdrivers.utils.param_utils import paramp
from cappy.config import dualhemtconfig, data2path
from cappy.balancer import Balancer

from qcodes.validators import Numbers, Callable, Validator
from qcodes.parameters import Parameter, ScaledParameter
import datetime

In [2]:
qc.Instrument.close_all()   # in case of existing connecitons
container = qc.Station(config_file=str(dualhemtconfig))

acdac = container.load_instrument("acdac")
dcdac = container.load_instrument("dcdac")
yoko = container.load_instrument("yoko")
keithley = container.load_instrument("keithley")
li1 = container.load_instrument("lockin")

print(100 * "-")
print(f"{datetime.datetime.now().strftime('%c')}")

Connected to: BARRERA ACDAC02 (serial:AD9106, firmware:2.00) in 3.08s
Connected to: BARRERA DCDAC01 (serial:AD5764, firmware:2.00) in 3.05s
Connected to: YOKOGAWA 765611 (serial:90ZC43353, firmware:1.10) in 0.03s
Connected to: KEITHLEY INSTRUMENTS DMM6500 (serial:04608216, firmware:1.7.12b) in 0.01s
Connected to: Stanford_Research_Systems SR865A (serial:005296, firmware:V1.51) in 0.07s
----------------------------------------------------------------------------------------------------
Thu Jan 30 14:44:34 2025


In [3]:
# Applied AC Voltages
Vref = acdac.ch1
V1pf = acdac.ch2
Vlref = acdac.ch3

# Applied DC Voltages
Vdd = yoko.channel1.voltage             # drives darin to source current across HEMT_A 
Vgamp = dcdac.channel1.voltage          # drives drain to source current across HEMT_R draining at HEMT_A gate 
Vgres = yoko.channel2.voltage           # gate voltage on HEMT_R 


# validators
non_positive_validator = Numbers(min_value=-2 , max_value=0)      
vg_max_validator = Numbers(min_value = -2, max_value=Vgamp.get())

# Vgamp < 0 and Vgres < Vgramp at all times 
Vgamp.add_validator(non_positive_validator)  
Vgres.add_validator(vg_max_validator)

# Measured Quantities 
Vdsdc = keithley.amplitude
Vdsac_R = li1.R
Vdsac_P = li1.P

In [16]:
def Vgamp_set(value):
    """
        Use this function to set Vgamp to value and update validator for Vgres. DO NOT set Vgamp natively. 
    """
    curr_Vgres = Vgres()
    if curr_Vgres > value:
        # update validator 
        vg_max_validator = Numbers(min_value = -3, max_value=value)
        Vgres.remove_validator()
        Vgres.add_validator(vg_max_validator)

        # set Vgamp and Vgres
        Vgres(value - 0.1)
        sleep(0.2)

    Vgamp(value)

def vgres_valid(val):
    vgamp = Vgamp()
    return vgamp - 0.5 <= val and val <=  vgamp 

# TODO:
numbertypes = float | int | np.floating | np.integer
class GateValidator(Validator[numbertypes]):
    """
    Requires a number  of type int, float, numpy.integer or numpy.floating.

    Args:
        min_value: Minimal value allowed, default -inf.
        max_value: Maximal value allowed, default inf.

    Raises:
        TypeError: If min or max value not a number. Or if min_value is
            larger than the max_value.

    """

    validtypes = (float, int, np.integer, np.floating)

    def __init__(self) -> None:
        if isinstance(min_value, self.validtypes):
            self._min_value = min_value
        else:
            raise TypeError("min_value must be a number")

        valuesok = max_value > min_value

        if isinstance(max_value, self.validtypes) and valuesok:
            self._max_value = max_value
        else:
            raise TypeError("max_value must be a number bigger than min_value")

        self._valid_values = (min_value, max_value)

    def validate(self, value: numbertypes, context: str = "") -> None:
        """
        Validate if number else raises error.

        Args:
            value: A number.
            context: Context for validation.

        Raises:
            TypeError: If not int or float.
            ValueError: If number is not between the min and the max value.

        """
        if not isinstance(value, self.validtypes):
            raise TypeError(f"{value!r} is not an int or float; {context}")

        if not (self._min_value <= value <= self._max_value):
            raise ValueError(
                f"{value!r} is invalid: must be between "
                f"{self._min_value} and {self._max_value} inclusive; {context}"
            )

    is_numeric = True

    def __repr__(self) -> str:
        minv = self._min_value if math.isfinite(self._min_value) else None
        maxv = self._max_value if math.isfinite(self._max_value) else None
        return "<Numbers{}>".format(range_str(minv, maxv, "v"))

    @property
    def min_value(self) -> float:
        return float(self._min_value)

    @property
    def max_value(self) -> float:
        return float(self._max_value)
    

In [53]:
acdac.display_mode("NORMAL")

In [30]:
def toggle_yoko(on=True):
    mode = 1 if on else 0
    yoko.channel1.output(mode)
    yoko.channel2.output(mode)

toggle_yoko()



In [58]:
Vgres(-0.1)

In [59]:
Vgres(-0.2)

In [60]:
Vgres(-0.3)

In [33]:
Vlref.voltage(200)
V1pf.voltage(100)
Vref.voltage(0)

In [39]:
Vgres.remove_validator()
Vgres.validators

(<Numbers -2<=v<=0>,)

In [40]:
Vgres(-0.5)

In [41]:
Vgamp(-0.4)

In [43]:
Vdd(0.1)

In [44]:
Vgres(-1)

In [46]:
Vgamp(-0.7)

In [47]:
paramp((Vgamp, Vdd))

In [48]:
paramp(Vgres)

In [49]:
Vgamp()

VisaIOError: ('VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.', "asking 'channel1:VOLTAGE?' to <DCDAC5764: dcdac>", 'getting dcdac_channel1_voltage')

In [ ]:
# measurement config for analysis 
indep_params = [Vgamp]
dep_params = [, Gout.R, Gout.P]
mc = {}
for param in indep_params + dep_params:
    mc[param.full_name] = param.label

s = sweep.Station(basedir=datapath, measurement_config=mc)
for param in indep_params + dep_params:
    s.fp(param)

s.add_comment("Vg sweep")

## Notes: 
    Vds (pin 20) wasn't reading voltage. Checked bonds and was disconnected. Rebonded and trying again. 

In [70]:
Vgres(0)
Vgres(), Vgamp(), Vdd()

(0.0, 0.0, 0.0)

floating pins

In [71]:
Vlref.voltage(200)
V1pf.voltage(100)
Vref.voltage(0)

In [72]:
Vdd(0.1)
Vgres(-0.5)

In [73]:
Vgamp(-0.4)

In [76]:
Vgres(-1)
yoko.print_readable_snapshot()

yoko:
	parameter      value
--------------------------------------------------------------------------------
IDN             :	{'vendor': 'YOKOGAWA', 'model': '765611', 'serial': '90ZC43...
display_settext :	None 
timeout         :	5 (s)
yoko_channel1:
	parameter    value
--------------------------------------------------------------------------------
auto_range    :	False 
current_limit :	1 (I)
four_wire     :	off 
output        :	True 
output_level  :	0.1 (V)
range         :	50 (V)
source_mode   :	VOLT 
system_errors :	0,"No error" 
voltage       :	0.1 (V)
voltage_limit :	1 (V)
voltage_range :	50 (V)
yoko_channel2:
	parameter    value
--------------------------------------------------------------------------------
auto_range    :	False 
current_limit :	1 (I)
four_wire     :	off 
output        :	True 
output_level  :	-1 (V)
range         :	50 (V)
source_mode   :	VOLT 
system_errors :	0,"No error" 
voltage       :	-1 (V)
voltage_limit :	1 (V)
voltage_range :	50 (V)


In [77]:
Vgres()

-1.0

In [78]:
Vgamp()

-0.400098

(-1.0, -0.400098)

In [81]:
Vgres(-1)

In [82]:
paramp(Vgamp)

In [84]:
Vgres(-0.6)

In [85]:
Vgres(-0.3)

In [86]:
Vgres(0)

In [87]:
paramp(Vdd)

**Log**: trying again,thought I blew hemt after trying to change the precision on the yoko current reading (ended up clicking wrong button and sending 1 V through Vgres). Paramped everything and checked current readings again -> nothing anomalous

In [92]:
acdac.reset()

In [4]:
Vdd(0.1)

In [5]:
paramp(Vdd)

In [6]:
Vdd(0.1)

In [7]:
Vgres(-0.5)

In [8]:
Vgamp(-0.4)

In [9]:
Vgres(-1)

In [11]:
indep_params = [Vgamp]
dep_params = [Vdsdc,Vdsac_R, Vdsac_P]

mc = {}
for param in indep_params + dep_params:
    mc[param.full_name] = param.label

s = sweep.Station(basedir=data2path,measurement_config=mc)
for param in indep_params + dep_params:
    s.fp(param)

s.add_comment("Vg sweep")

for dep in dep_params:
    s.plot(Vgamp, dep)

In [13]:
START = -0.4
STOP = -0.9
SAMPLES = 100
INT_TIME = 1.0

setpoints = np.linspace(START, STOP, SAMPLES)
res = s.sweep(Vgamp,setpoints, delay=INT_TIME)

2025-01-30 14:57:45,254 [INFO] Starting sweep with ID 1
2025-01-30 14:57:45,255 [INFO] Minimum duration 0h 1m 40s


  0%|          | 0/100 [00:00<?, ?it/s]

VisaIOError: ('VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.', "asking 'channel1:VOLTAGE?' to <DCDAC5764: dcdac>", 'getting dcdac_channel1_voltage')

In [14]:
qc.Instrument.close(dcdac)

In [15]:
dcdac = container.load_instrument("dcdac")

[dcdac(DCDAC5764)] Error getting or interpreting *IDN?: ''
Traceback (most recent call last):
  File "C:\Users\barreralab\AppData\Local\Programs\Python\Python312\Lib\site-packages\qcodes\instrument\instrument.py", line 95, in get_idn
    idstr = self.ask("*IDN?")
            ^^^^^^^^^^^^^^^^^
  File "C:\Users\barreralab\AppData\Local\Programs\Python\Python312\Lib\site-packages\qcodes\instrument\instrument.py", line 439, in ask
    raise e
  File "C:\Users\barreralab\AppData\Local\Programs\Python\Python312\Lib\site-packages\qcodes\instrument\instrument.py", line 432, in ask
    answer = self.ask_raw(cmd)
             ^^^^^^^^^^^^^^^^^
  File "C:\Users\barreralab\AppData\Local\Programs\Python\Python312\Lib\site-packages\qcodes\instrument\visa.py", line 390, in ask_raw
    response = self.visa_handle.query(cmd)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\barreralab\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyvisa\resources\messagebased.py", line 647, in que

Connected to: None dcdac (serial:None, firmware:None) in 8.01s


[dcdac_channel1(DCDAC5764Channel)] Snapshot: Could not update parameter: voltage
[dcdac_channel2(DCDAC5764Channel)] Snapshot: Could not update parameter: voltage
[dcdac_channel3(DCDAC5764Channel)] Snapshot: Could not update parameter: voltage
[dcdac_channel4(DCDAC5764Channel)] Snapshot: Could not update parameter: voltage
[dcdac_channel5(DCDAC5764Channel)] Snapshot: Could not update parameter: voltage
[dcdac_channel6(DCDAC5764Channel)] Snapshot: Could not update parameter: voltage
[dcdac_channel7(DCDAC5764Channel)] Snapshot: Could not update parameter: voltage


Received SIGINT, Will interrupt at first suitable time. Send second SIGINT to interrupt immediately.


KeyboardInterrupt: 

In [17]:
paramp(Vgres)

In [18]:
paramp(Vdd)

In [19]:
acdac.reset()